In [10]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key


ImportError: cannot import name 'g_key' from 'config' (c:\Users\usvisin3\OneDrive - PG (1)\12 Personal\Data Analytics Learning\World_Weather_Analysis\config.py)